In [55]:
!nvidia-smi

Fri Dec 24 09:37:18 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   36C    P8    27W / 149W |      3MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [56]:
import numpy as np
import torch

In [57]:
#making training data 
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70]], dtype='float32')

In [58]:
# Targets (apples, oranges)
target = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119]], dtype='float32')

In [59]:
#Convert input and target to tensors
inputs = torch.from_numpy(inputs)
target = torch.from_numpy(target)

print(inputs,"\n")
print(target)

tensor([[ 73.,  67.,  43.],
        [ 91.,  88.,  64.],
        [ 87., 134.,  58.],
        [102.,  43.,  37.],
        [ 69.,  96.,  70.]]) 

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [60]:
# weights and biases
w = torch.randn(2,3 , requires_grad=True)
b = torch.randn(2, requires_grad=True)

print(w)
print(b)

tensor([[-0.7145,  1.2718, -0.1902],
        [-0.7467,  1.0642,  0.6049]], requires_grad=True)
tensor([-0.7670,  0.0596], requires_grad=True)


In [61]:
#define the model

def model(x):
  return x @ w.t() + b

In [62]:
# prediction
preds = model(inputs)
print(preds)

tensor([[ 24.1113,  42.8640],
        [ 33.9653,  64.4753],
        [ 96.4693, 112.7846],
        [-25.9916,  -7.9600],
        [ 58.7174,  93.0456]], grad_fn=<AddBackward0>)


In [63]:
#actual
print(target)

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])


In [64]:
# loss function MSE
def MSE(actual, target):
  diff = actual - target
  return torch.sum(diff * diff) / diff.numel()


In [65]:
# error
loss = MSE(target, preds)
print(loss)

tensor(1317.5035, grad_fn=<DivBackward0>)


In [66]:
# compute gradients
loss.backward()

In [67]:
print(w, "\n")
print(w.grad)

tensor([[-0.7145,  1.2718, -0.1902],
        [-0.7467,  1.0642,  0.6049]], requires_grad=True) 

tensor([[-3303.7683, -3121.8945, -2112.7368],
        [-2688.0376, -2433.2100, -1631.4502]])


In [68]:
print(b, "\n")
print(b.grad)

tensor([-0.7670,  0.0596], requires_grad=True) 

tensor([-38.7457, -30.9581])


In [69]:
#reset grad
w.grad.zero_()
b.grad.zero_()

print(w.grad)
print(b.grad)

tensor([[0., 0., 0.],
        [0., 0., 0.]])
tensor([0., 0.])


In [70]:
# adjust params

preds = model(inputs)
print(preds)

tensor([[ 24.1113,  42.8640],
        [ 33.9653,  64.4753],
        [ 96.4693, 112.7846],
        [-25.9916,  -7.9600],
        [ 58.7174,  93.0456]], grad_fn=<AddBackward0>)


In [71]:
# loss
loss = MSE(target, preds)
print(loss)

tensor(1317.5035, grad_fn=<DivBackward0>)


In [72]:
loss.backward()

print(w.grad, "\n")
print(b.grad)


tensor([[-3303.7683, -3121.8945, -2112.7368],
        [-2688.0376, -2433.2100, -1631.4502]]) 

tensor([-38.7457, -30.9581])


In [73]:
  # adjust weight & reset grad
with torch.no_grad():
    w -= w.grad * 1e-5
    b -= b.grad * 1e-5
    w.grad.zero_()
    b.grad.zero_()

In [74]:
print(w)
print(b)

tensor([[-0.6814,  1.3031, -0.1691],
        [-0.7198,  1.0885,  0.6213]], requires_grad=True)
tensor([-0.7666,  0.0599], requires_grad=True)


In [75]:
# calculate again
preds = model(inputs)
loss = MSE(target, preds)
print(loss)

tensor(944.6516, grad_fn=<DivBackward0>)


In [76]:
# Training for multiple epochs
for i in range(400):
  preds = model(inputs)
  loss = MSE(target, preds)
  loss.backward()

  with torch.no_grad():
     w -= w.grad * 1e-5 # learning rate
     b -= b.grad * 1e-5
     w.grad.zero_()
     b.grad.zero_()
  print(f"Epochs({i}/{100}) & Loss {loss}")

Epochs(0/100) & Loss 944.6515502929688
Epochs(1/100) & Loss 692.81640625
Epochs(2/100) & Loss 522.5397338867188
Epochs(3/100) & Loss 407.2310485839844
Epochs(4/100) & Loss 328.97125244140625
Epochs(5/100) & Loss 275.68511962890625
Epochs(6/100) & Loss 239.23495483398438
Epochs(7/100) & Loss 214.13687133789062
Epochs(8/100) & Loss 196.69528198242188
Epochs(9/100) & Loss 184.41964721679688
Epochs(10/100) & Loss 175.63180541992188
Epochs(11/100) & Loss 169.2005157470703
Epochs(12/100) & Loss 164.36334228515625
Epochs(13/100) & Loss 160.6066436767578
Epochs(14/100) & Loss 157.58407592773438
Epochs(15/100) & Loss 155.06198120117188
Epochs(16/100) & Loss 152.8831329345703
Epochs(17/100) & Loss 150.94161987304688
Epochs(18/100) & Loss 149.1654510498047
Epochs(19/100) & Loss 147.5065460205078
Epochs(20/100) & Loss 145.9321746826172
Epochs(21/100) & Loss 144.4202117919922
Epochs(22/100) & Loss 142.95587158203125
Epochs(23/100) & Loss 141.52902221679688
Epochs(24/100) & Loss 140.13259887695312
E

In [77]:
preds = model(inputs)
loss = MSE(target, preds)
print(loss)

tensor(20.2836, grad_fn=<DivBackward0>)


In [78]:
from math import sqrt
sqrt(loss)

4.503729016935122

In [79]:
preds

tensor([[ 57.2541,  70.1743],
        [ 77.9259,  99.1314],
        [128.2841, 136.6749],
        [ 21.9018,  36.0246],
        [ 93.8439, 117.0515]], grad_fn=<AddBackward0>)

In [80]:
target

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.]])

# Linear Regression using PyTorch built-ins

In [81]:
import torch.nn as nn

In [82]:
# Input (temp, rainfall, humidity)
inputs = np.array([[73, 67, 43], 
                   [91, 88, 64], 
                   [87, 134, 58], 
                   [102, 43, 37], 
                   [69, 96, 70], 
                   [74, 66, 43], 
                   [91, 87, 65], 
                   [88, 134, 59], 
                   [101, 44, 37], 
                   [68, 96, 71], 
                   [73, 66, 44], 
                   [92, 87, 64], 
                   [87, 135, 57], 
                   [103, 43, 36], 
                   [68, 97, 70]], 
                  dtype='float32')


In [83]:
inputs.shape

(15, 3)

In [84]:
# Targets (apples, oranges)
targets = np.array([[56, 70], 
                    [81, 101], 
                    [119, 133], 
                    [22, 37], 
                    [103, 119],
                    [57, 69], 
                    [80, 102], 
                    [118, 132], 
                    [21, 38], 
                    [104, 118], 
                    [57, 69], 
                    [82, 100], 
                    [118, 134], 
                    [20, 38], 
                    [102, 120]], 
                   dtype='float32')

In [85]:
inputs = torch.from_numpy(inputs)
targets = torch.from_numpy(targets)

In [86]:
print(inputs.dtype)
print(targets.dtype)

torch.float32
torch.float32


In [87]:
# Dataset & DataLoader
from torch.utils.data import TensorDataset

In [88]:
#define dataset
train_ds = TensorDataset(inputs, targets)

In [89]:
type(train_ds)

torch.utils.data.dataset.TensorDataset

In [90]:
train_ds[0:3]

(tensor([[ 73.,  67.,  43.],
         [ 91.,  88.,  64.],
         [ 87., 134.,  58.]]), tensor([[ 56.,  70.],
         [ 81., 101.],
         [119., 133.]]))

In [91]:
from torch.utils.data import DataLoader

In [92]:
batch_size = 5
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)

In [93]:
type(train_dl)

torch.utils.data.dataloader.DataLoader

In [94]:
for xb, yb in train_dl:
  print(xb)
  print(yb)
  break

tensor([[ 69.,  96.,  70.],
        [ 68.,  97.,  70.],
        [ 73.,  66.,  44.],
        [ 68.,  96.,  71.],
        [103.,  43.,  36.]])
tensor([[103., 119.],
        [102., 120.],
        [ 57.,  69.],
        [104., 118.],
        [ 20.,  38.]])


In [95]:
#define model
model = nn.Linear(3,2)
print(model.weight)
print(model.bias)

Parameter containing:
tensor([[ 0.0333,  0.3485, -0.1712],
        [-0.2529,  0.2025,  0.1221]], requires_grad=True)
Parameter containing:
tensor([-0.4201,  0.1001], requires_grad=True)


In [96]:
list(model.parameters())

[Parameter containing:
 tensor([[ 0.0333,  0.3485, -0.1712],
         [-0.2529,  0.2025,  0.1221]], requires_grad=True),
 Parameter containing:
 tensor([-0.4201,  0.1001], requires_grad=True)]

In [97]:
preds = model(inputs)
preds

tensor([[ 1.8005e+01,  4.5854e-01],
        [ 2.2329e+01,  2.7241e+00],
        [ 3.9256e+01,  1.2319e+01],
        [ 1.1633e+01, -1.2470e+01],
        [ 2.3357e+01,  1.0642e+01],
        [ 1.7689e+01,  3.0818e-03],
        [ 2.1810e+01,  2.6437e+00],
        [ 3.9118e+01,  1.2189e+01],
        [ 1.1948e+01, -1.2014e+01],
        [ 2.3153e+01,  1.1017e+01],
        [ 1.7485e+01,  3.7815e-01],
        [ 2.2014e+01,  2.2686e+00],
        [ 3.9776e+01,  1.2400e+01],
        [ 1.1837e+01, -1.2845e+01],
        [ 2.3673e+01,  1.1097e+01]], grad_fn=<AddmmBackward0>)

In [98]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])

In [99]:
#loss function
import torch.nn.functional as F

In [100]:
loss_fn  = F.mse_loss

In [101]:

# ??nn.Linear

In [102]:
loss = loss_fn(model(inputs), targets)
print(loss)

tensor(6087.3535, grad_fn=<MseLossBackward0>)


In [103]:
#optimizers
opt = torch.optim.SGD(model.parameters(), lr= 1e-5)

In [104]:
def fit(num_epochs, model, loss_fn, opt):
  for epoch in range(1,num_epochs+1):
    for xb, yb in train_dl:
      pred = model(xb)
      loss = loss_fn(pred, yb)
      loss.backward()
      opt.step()
      opt.zero_grad()
    
    if(epoch) % 10 ==0:
      print(f'Epoch [{epoch}/{num_epochs}], Loss {loss.item()}')



In [105]:
fit(100, model, loss_fn, opt)

Epoch [10/100], Loss 146.14207458496094
Epoch [20/100], Loss 333.1202697753906
Epoch [30/100], Loss 67.80695343017578
Epoch [40/100], Loss 79.31674194335938
Epoch [50/100], Loss 33.19841766357422
Epoch [60/100], Loss 76.24372100830078
Epoch [70/100], Loss 21.04210090637207
Epoch [80/100], Loss 44.46441650390625
Epoch [90/100], Loss 36.90877151489258
Epoch [100/100], Loss 27.580245971679688


In [106]:
preds = model(inputs)
preds

tensor([[ 58.1973,  71.2937],
        [ 79.2595,  98.5598],
        [123.1435, 136.3076],
        [ 27.5918,  42.2535],
        [ 92.9538, 112.4204],
        [ 57.0212,  70.2557],
        [ 78.5153,  98.2359],
        [123.1519, 136.7040],
        [ 28.7678,  43.2916],
        [ 93.3857, 113.1346],
        [ 57.4532,  70.9698],
        [ 78.0834,  97.5217],
        [123.8876, 136.6315],
        [ 27.1598,  41.5394],
        [ 94.1298, 113.4585]], grad_fn=<AddmmBackward0>)

In [107]:
targets

tensor([[ 56.,  70.],
        [ 81., 101.],
        [119., 133.],
        [ 22.,  37.],
        [103., 119.],
        [ 57.,  69.],
        [ 80., 102.],
        [118., 132.],
        [ 21.,  38.],
        [104., 118.],
        [ 57.,  69.],
        [ 82., 100.],
        [118., 134.],
        [ 20.,  38.],
        [102., 120.]])